<a href="https://colab.research.google.com/github/esther-pui/WOA7015-A2/blob/main/WOA7015_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Logistic regression from scratch**

In [ ]:
import numpy as np
train_data = np.loadtxt('/Train_toydata.txt')
test_data = np.loadtxt('/Test_toydata.txt')

x_features = train_data[:, :-1]
y_class = train_data[:, -1].reshape(-1, 1)

m = x_features.shape[0]
bias_col = np.ones((m, 1))

# n x 3 matrix, Bias + Features
x_train_augmented = np.hstack((bias_col, x_features))

def sigmoid(z):
  return 1 / (1 + np.exp(-z))


# **Function 1**

In [ ]:
def CalcObj(XTrain, YTrain, wHat):

  n_size = XTrain.shape[0]

  # weight + bias
  z = np.dot(XTrain, wHat)
  g = sigmoid(z)
  g_clipped = np.clip(g, 1e-15, 1 - 1e-15)

  #class 1
  term1 = np.multiply(YTrain, np.log(g_clipped))

  #class 0
  term2 = np.multiply(1 - YTrain, np.log(1 - g_clipped))
  obj = -(1 / n_size) * np.sum(term1 + term2)

  return obj

# **Function 2**

In [ ]:
def CalcGrad(XTrain, YTrain, wHat):

  n_size = XTrain.shape[0]
  z = np.dot(XTrain, wHat)
  h_sigmoid = sigmoid(z)

  # difference between true vector and prediction
  Y_error_vector = h_sigmoid - YTrain

  # matrix multiplication
  grad = np.dot(XTrain.T, Y_error_vector) / n_size

  return grad

# **Function 3**

In [ ]:
def UpdateParams(weight, grad, lr):
  new_w = weight - (lr * grad)
  return new_w

# **Function 4**

In [ ]:
def CheckConvg(oldObj, newObj, tol):
  has_converged = np.abs(oldObj - newObj) < tol
  return has_converged

# **Function 5**

In [ ]:
LR = 0.01
TOL = 0.001

def GradientDescent(XTrain, YTrain):

  parameter_size = XTrain.shape[1]
  # starting weight & bias
  wHat = np.zeros((parameter_size, 1))

  objVals = []
  converged = False

  # initial loss, store scalar for comparison
  current_loss_scalar = CalcObj(XTrain, YTrain, wHat).item()
  objVals.append(current_loss_scalar)

  i = 0

  while not converged:
    old_loss_scalar = current_loss_scalar

    #training
    grad = CalcGrad(XTrain, YTrain, wHat)
    wHat = UpdateParams(wHat, grad, LR)

    #calculate new loss
    new_loss_scalar = CalcObj(XTrain, YTrain, wHat).item()
    objVals.append(new_loss_scalar)

    #check converge
    converged = CheckConvg(old_loss_scalar, new_loss_scalar, TOL)
    current_loss_scalar = new_loss_scalar
    i += 1

  return wHat, objVals


# **Training**

In [ ]:
final_wHat, loss_history = GradientDescent(x_train_augmented, y_class)

# **Function 6**

In [ ]:

def PredictLabels (XTest, YTest, wHat):

  # forward pass 1
  z = np.dot(XTest, wHat)

  # forward pass 2
  h = sigmoid(z)

  # classification
  yHat = np.where(h >= 0.5, 1, 0)

  # error count
  misclassified_idx = np.where(np.not_equal(yHat, YTest))[0]
  numErrors = len(misclassified_idx)

  if numErrors > 0:
    first_error_idx = misclassified_idx[0]
    predicted_val = yHat[first_error_idx, 0]
    actual_val = YTest[first_error_idx, 0]

    print("\nMisclassified samples:")
    print(f"Index: {first_error_idx}, Predicted: {predicted_val}, Actual: {actual_val}")

  return yHat, numErrors


# **Predict**

In [ ]:
x_test_features = test_data[:, :-1]
y_test_class = test_data[:, -1].reshape(-1, 1)

# add Bias Term
m_test = x_test_features.shape[0]
bias_col_test = np.ones((m_test, 1))
x_test_augmented = np.hstack((bias_col_test, x_test_features))

yHat, numErrors = PredictLabels(x_test_augmented, y_test_class, final_wHat)

# get accuracy
total_samples = y_test_class.shape[0]
accuracy_scratch = (total_samples - numErrors) / total_samples

print("\n---- Scratch result ----")
print(f"Total test samples: {total_samples}")
print(f"Total misclassified errors: {numErrors}")
print(f"Accuracy: {accuracy_scratch * 100:.2f}%")



Misclassified samples:
Index: 2, Predicted: 1, Actual: 0.0

---- Scratch result ----
Total test samples: 20
Total misclassified errors: 1
Accuracy: 95.00%


# **Logistic regression using PyTorch**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# convert to tensor
def to_tensor(x, y):
    x_tensor = torch.tensor(x, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)
    return x_tensor, y_tensor

x_train_tensor, y_train_tensor = to_tensor(x_train_augmented, y_class)
x_test_tensor, y_test_tensor = to_tensor(x_test_augmented, y_test_class)

# model class
class LogisticRegression(nn.Module):
  def __init__(self, input_size):
    super(LogisticRegression, self).__init__()

    # initiate parameter
    self.linear = nn.Linear(input_size, 1, bias=False)

  # get prediction outputs
  def forward(self, x):
    return self.linear(x)

model = LogisticRegression(input_size=x_train_augmented.shape[1])

# GradientDescent
optimizer = optim.SGD(model.parameters(), lr=0.01) #same lr as task 1

# CalcObj
loss_function = nn.BCEWithLogitsLoss()

# training loop
num_epochs = 5000
for epoch in range(num_epochs):

  # run forward
  outputs = model(x_train_tensor)
  loss = loss_function(outputs, y_train_tensor)

  # reset
  optimizer.zero_grad()

  # CalcGrad
  loss.backward()

  # UpdateParams
  optimizer.step()

# testing and get acuracy
with torch.no_grad(): # no gradient
  outputs = model(x_test_tensor)
  predicted_probs = torch.sigmoid(outputs)
  y_predicted_tensor = (predicted_probs >= 0.5).float()

  total_samples_pytorch = y_test_tensor.size(0)
  numErrors_pytorch = (y_predicted_tensor != y_test_tensor).sum().item()
  accuracy_pytorch = (total_samples_pytorch - numErrors_pytorch) / total_samples_pytorch

# result
print("\n---- PyTorch result ----")
print(f"Total test samples: {total_samples_pytorch}")
print(f"Total misclassified errors: {numErrors_pytorch}")
print(f"Accuracy: {accuracy_pytorch * 100:.2f}%")

# find misclassified samples
misclassified_indices_pytorch = (y_predicted_tensor != y_test_tensor).nonzero(as_tuple=True)[0]

print("\nMisclassified samples:")
for idx in misclassified_indices_pytorch:
    print(f"Index: {idx.item()}, Predicted: {y_predicted_tensor[idx].item()}, Actual: {y_test_tensor[idx].item()}")



---- PyTorch result ----
Total test samples: 20
Total misclassified errors: 1
Accuracy: 95.00%

Misclassified samples:
Index: 2, Predicted: 1.0, Actual: 0.0
